In [1]:
from SYK_fft import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [20]:
# Nbig = int(1e4)
Nbig = 10
Jot = 1.0
err = 1e-4

beta = np.pi
beta = 10
mu = 0
g = 1.0
J = 1.0
phase = np.pi/2
lamb = 1.0 * np.exp(1.0j * phase)
alpha = 1.0


omega = (2 * np.arange(Nbig) - Nbig + 1) * np.pi/beta
nu = (2 * np.arange(Nbig) - Nbig ) * np.pi/beta
tau = (np.arange(Nbig) + 1./2) * beta/Nbig

In [ ]:
GDtau = np.ones(Nbig)
#SigmaDtau = np.ones(Nbig)
GODtau = np.ones(Nbig)
#SigmaODtau = np.ones(Nbig)
DDtau = np.ones(Nbig)
#Pitau = np.ones(Nbig)
DODtau = np.ones(Nbig)



itern = 0
diff = 1
x = 0.5



while(diff>err):
    itern+=1
    diffold = diff
    
    oldGDtau = 1.0*GDtau
    oldGODtau = 1.0*GODtau
    oldDDtau = 1.0*DDtau
    oldDODtau = 1.0*DODtau
    
    Goldomega = Time2FreqF(Goldtau,Nbig,beta)
    
    
    SigmaDtau = kappa * (g**2) * DDtau * GDtau
    Sigmaomega = Time2FreqF(Sigmatau,Nbig,beta)
    Gomega = x/(-1j*omega - Sigmaomega) + (1-x)*Goldomega
    Gtau = Freq2TimeF(Gomega,Nbig,beta)
    diff = (1/Nbig) * np. sqrt(np.sum((np.abs(Gtau-Goldtau))**2))
    if diff>diffold:
        x/=2





    
print("iter = ",itern," , diff = ", diff)
    

In [ ]:

val = 1

ax = plt.plot(np.real(Gtau[val:-val]),'r')

plt.xlim([0,Nbig])
plt.ylim([0,0.6])
plt.title("G(tau) ")
plt.show()

In [8]:
A = np.array([1,2,3,4,5])
A * A[::-1] 

array([5, 8, 9, 8, 5])

In [15]:
nu, omega

(array([-6., -4., -2.,  0.,  2.,  4.]), array([-5., -3., -1.,  1.,  3.,  5.]))

In [21]:
tau

array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5])

In [25]:
i = 0 
tau[i], beta-tau[i], tau[-i-1]

(0.5, 9.5, 9.5)